In [1]:
!pip install catboost

     |████████████████████████████████| 65.9MB 61kB/s 


In [2]:
#baseline recommend_syst_pro by Alex-Tele2-Data team
import numpy as np
import pandas as pd  #for dataframes
from sklearn.model_selection import train_test_split  #to divide data into train and validate
from sklearn.metrics import roc_auc_score  #for metric
from catboost import CatBoostClassifier
import pickle #for saving model
RS=42  #we will fix random state

In [3]:
#let s connect to google drive with data
from google.colab import drive
drive.mount ('/content/gdrive', force_remount = True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [4]:
!ls '../content/gdrive/My Drive/dataset_MADE_2020_taskB'

item-features.csv  test.csv  train.csv	user-features.csv


In [5]:
data_df=pd.read_csv('../content/gdrive/My Drive/dataset_MADE_2020_taskB/train.csv',header=0)
data_df.head()

,user_id,item_id,like,timestamp
0,140,342,0,1490936622
1,378,172,1,1490936628
2,150,182,0,1490936650
3,455,17,0,1490936704
4,350,409,0,1490936735


In [6]:
test_df=pd.read_csv('../content/gdrive/My Drive/dataset_MADE_2020_taskB/test.csv',
                      header=0)
test_df.head()

,user_id,timestamp
0,166,1490944431
1,26,1490957371
2,41,1490958147
3,286,1490971255
4,108,1490976836


In [7]:
data_df.describe()

,user_id,item_id,like,timestamp
count,8674.000000,8674.000000,8674.000000,8.674000e+03
mean,244.855891,179.805626,0.175928,1.491073e+09
std,143.102186,126.366142,0.380781,7.970218e+04
min,0.000000,0.000000,0.000000,1.490937e+09
25%,121.000000,73.000000,0.000000,1.491004e+09
50%,243.000000,154.000000,0.000000,1.491071e+09
75%,368.000000,266.000000,0.000000,1.491141e+09
max,496.000000,443.000000,1.000000,1.491216e+09


In [8]:
data_df['item_id'].nunique()

444

In [9]:
#divide dataset to train and test
df_x, x_val, df_y, y_val=train_test_split(data_df[['user_id','item_id','timestamp']],\
                                                data_df['like'], test_size=0.05,random_state=RS)

In [10]:
#pl_clf = RandomForestClassifier(n_estimators=200,max_depth=300,
#                                                            n_jobs=-1, \
#                                                  verbose=True,random_state=RS)
#pl_clf = LGBMClassifier(n_estimators=200,max_depth=200,
#                        n_jobs=-1,
#                        silent=False,
#                        random_state=RS)
pl_clf = CatBoostClassifier(iterations=1500,
                        task_type="GPU",
                        depth=8,
                        learning_rate=0.1,
                        random_seed=RS)

In [11]:
pl_clf.fit(df_x,df_y,eval_set=(x_val,y_val));

0:	learn: 0.5593764	test: 0.5664847	best: 0.5664847 (0)	total: 129ms	remaining: 3m 12s
1:	learn: 0.5014394	test: 0.5138015	best: 0.5138015 (1)	total: 247ms	remaining: 3m 4s
2:	learn: 0.4461295	test: 0.4619392	best: 0.4619392 (2)	total: 324ms	remaining: 2m 41s
3:	learn: 0.4165291	test: 0.4346733	best: 0.4346733 (3)	total: 447ms	remaining: 2m 47s
4:	learn: 0.3930198	test: 0.4137608	best: 0.4137608 (4)	total: 565ms	remaining: 2m 48s
5:	learn: 0.3761317	test: 0.3983975	best: 0.3983975 (5)	total: 683ms	remaining: 2m 49s
6:	learn: 0.3580076	test: 0.3827578	best: 0.3827578 (6)	total: 803ms	remaining: 2m 51s
7:	learn: 0.3407416	test: 0.3675026	best: 0.3675026 (7)	total: 920ms	remaining: 2m 51s
8:	learn: 0.3331594	test: 0.3601851	best: 0.3601851 (8)	total: 1.04s	remaining: 2m 51s
9:	learn: 0.3258308	test: 0.3541090	best: 0.3541090 (9)	total: 1.16s	remaining: 2m 52s
10:	learn: 0.3183997	test: 0.3468943	best: 0.3468943 (10)	total: 1.27s	remaining: 2m 52s
11:	learn: 0.3115436	test: 0.3411200	best:

In [12]:
print('Скор на трейне',pl_clf.score(df_x, df_y))
print('Скор по валидации',pl_clf.score(x_val, y_val))

Скор на трейне 0.9226941747572815
Скор по валидации 0.8824884792626728


In [14]:
#let s save our model to use on server
filename = 'model_recommend_cb01.pickle'
pickle.dump(pl_clf, open(filename, 'wb'))

# This is example of code how to load saved model
# load the model from disk
# loaded_model = pickle.load(open(filename, 'rb'))
# prediction = loaded_model.predict(X_test)
# print(prediction)

In [15]:
#let s save model from colaboratory oto our PC
from google.colab import files
files.download('model_recommend_cb01.pickle')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [16]:
#take most popular 300 items
top_items=data_df['item_id'].value_counts(ascending=False)[:300]
print(top_items)

100    110
143    100
194     98
390     98
391     98
      ... 
186      7
376      7
360      7
352      7
380      7
Name: item_id, Length: 300, dtype: int64


In [17]:
print(top_items[:20])

100    110
143    100
194     98
390     98
391     98
76      97
403     97
402     96
405     95
404     95
178     93
118     88
154     85
165     85
9       80
129     80
22      77
35      76
92      71
44      66
Name: item_id, dtype: int64


In [18]:
lst_top20=[100,143,194,390,391,76,403,402,405,404,178,118,154,165,9,129,22,35,92,44]

In [19]:
data_df['like'].value_counts()

0    7148
1    1526
Name: like, dtype: int64

In [20]:
lst_out=[]  #list for resul list
N_items=20  #number of predicted items for user
for index, row in test_df.iterrows():
  temp_user=row[0]
  temp_time=row[1]
  cnt=0
  temp_lst=[]
  for i in top_items:
    pred=pl_clf.predict([temp_user,i,temp_time])
    print(temp_user,i,temp_time,pred)
    if pred:
      cnt+=1
      temp_lst.append(i)
      if cnt > N_items:
        break
  temp_set=set(temp_lst)
  temp_lst2=list(temp_set)  #избавляемся от дублей, откуда они взялись непонятно
  if len(temp_lst2) < 20:
    temp_lst2.extend(lst_top20[:20-len(temp_lst2)])
  lst_out.append(temp_lst2)
      







Выходные данные были обрезаны до нескольких последних строк (5000).
278 28 1491212375 0
278 28 1491212375 0
278 27 1491212375 0
278 27 1491212375 0
278 26 1491212375 0
278 26 1491212375 0
278 26 1491212375 0
278 25 1491212375 0
278 25 1491212375 0
278 25 1491212375 0
278 25 1491212375 0
278 25 1491212375 0
278 24 1491212375 0
278 24 1491212375 0
278 24 1491212375 0
278 24 1491212375 0
278 23 1491212375 0
278 23 1491212375 0
278 23 1491212375 0
278 22 1491212375 0
278 22 1491212375 0
278 22 1491212375 0
278 21 1491212375 0
278 21 1491212375 0
278 20 1491212375 0
278 20 1491212375 0
278 20 1491212375 0
278 20 1491212375 0
278 20 1491212375 0
278 20 1491212375 0
278 20 1491212375 0
278 20 1491212375 0
278 20 1491212375 0
278 19 1491212375 0
278 19 1491212375 0
278 19 1491212375 0
278 19 1491212375 0
278 19 1491212375 0
278 19 1491212375 0
278 19 1491212375 0
278 19 1491212375 0
278 19 1491212375 0
278 19 1491212375 0
278 18 1491212375 0
278 18 1491212375 0
278 18 1491212375 0
278 18 14912

In [22]:
lst_out=[]  #list for resul list
N_items=20  #number of predicted items for user
for index, row in test_df.iterrows():
  temp_user=row[0]
  temp_time=row[1]
  cnt=0
  temp_lst=[]
  for i in top_items:
    pred=pl_clf.predict([temp_user,i,temp_time])
    print(temp_user,i,temp_time,pred)
    if pred:
      cnt+=1
      temp_lst.append(i)
      if cnt > N_items:
        break
  temp_set=set(temp_lst)
  temp_lst2=list(temp_set)  
  if len(temp_lst) < 20:
    temp_lst.extend(lst_top20[:20-len(temp_lst)])
  lst_out.append(temp_lst)
      


Выходные данные были обрезаны до нескольких последних строк (5000).
278 28 1491212375 0
278 28 1491212375 0
278 27 1491212375 0
278 27 1491212375 0
278 26 1491212375 0
278 26 1491212375 0
278 26 1491212375 0
278 25 1491212375 0
278 25 1491212375 0
278 25 1491212375 0
278 25 1491212375 0
278 25 1491212375 0
278 24 1491212375 0
278 24 1491212375 0
278 24 1491212375 0
278 24 1491212375 0
278 23 1491212375 0
278 23 1491212375 0
278 23 1491212375 0
278 22 1491212375 0
278 22 1491212375 0
278 22 1491212375 0
278 21 1491212375 0
278 21 1491212375 0
278 20 1491212375 0
278 20 1491212375 0
278 20 1491212375 0
278 20 1491212375 0
278 20 1491212375 0
278 20 1491212375 0
278 20 1491212375 0
278 20 1491212375 0
278 20 1491212375 0
278 19 1491212375 0
278 19 1491212375 0
278 19 1491212375 0
278 19 1491212375 0
278 19 1491212375 0
278 19 1491212375 0
278 19 1491212375 0
278 19 1491212375 0
278 19 1491212375 0
278 19 1491212375 0
278 18 1491212375 0
278 18 1491212375 0
278 18 1491212375 0
278 18 14912

In [23]:

print(lst_out[:5])

[[88, 80, 80, 77, 76, 71, 66, 65, 59, 58, 58, 39, 39, 39, 37, 37, 36, 36, 36, 36, 36], [88, 85, 85, 80, 80, 77, 76, 71, 66, 65, 62, 61, 61, 59, 58, 58, 39, 39, 39, 37, 37], [88, 85, 85, 80, 80, 77, 76, 66, 65, 62, 61, 61, 59, 58, 58, 39, 39, 39, 37, 37, 36], [80, 80, 77, 76, 66, 65, 59, 58, 58, 37, 37, 36, 36, 36, 36, 36, 36, 35, 35, 35, 35], [88, 80, 80, 77, 76, 66, 65, 62, 61, 61, 59, 58, 58, 39, 39, 39, 37, 37, 36, 36, 36]]


In [24]:
lst_0=[]   #list for results
lst_1=[]
lst_2=[]
lst_3=[]
lst_4=[]
lst_5=[]
lst_6=[]
lst_7=[]
lst_8=[]
lst_9=[]
lst_10=[]
lst_11=[]
lst_12=[]
lst_13=[]
lst_14=[]
lst_15=[]
lst_16=[]
lst_17=[]
lst_18=[]
lst_19=[]
for j in lst_out:
  lst_0.append(j[0])
  lst_1.append(j[1])
  lst_2.append(j[2])
  lst_3.append(j[3])
  lst_4.append(j[4])
  lst_5.append(j[5])
  lst_6.append(j[6])
  lst_7.append(j[7])
  lst_8.append(j[8])
  lst_9.append(j[9])
  lst_10.append(j[10])
  lst_11.append(j[11])
  lst_12.append(j[12])
  lst_13.append(j[13])
  lst_14.append(j[14])
  lst_15.append(j[15])
  lst_16.append(j[16])
  lst_17.append(j[17])
  lst_18.append(j[18])
  lst_19.append(j[19])


In [25]:
test_df['0']=lst_0
test_df['1']=lst_1
test_df['2']=lst_2
test_df['3']=lst_3
test_df['4']=lst_4
test_df['5']=lst_5
test_df['6']=lst_6
test_df['7']=lst_7
test_df['8']=lst_8
test_df['9']=lst_9
test_df['10']=lst_10
test_df['11']=lst_11
test_df['12']=lst_12
test_df['13']=lst_13
test_df['14']=lst_14
test_df['15']=lst_15
test_df['16']=lst_16
test_df['17']=lst_17
test_df['18']=lst_18
test_df['19']=lst_19

test_df.head()

,user_id,timestamp,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,166,1490944431,88,80,80,77,76,71,66,65,59,58,58,39,39,39,37,37,36,36,36,36
1,26,1490957371,88,85,85,80,80,77,76,71,66,65,62,61,61,59,58,58,39,39,39,37
2,41,1490958147,88,85,85,80,80,77,76,66,65,62,61,61,59,58,58,39,39,39,37,37
3,286,1490971255,80,80,77,76,66,65,59,58,58,37,37,36,36,36,36,36,36,35,35,35
4,108,1490976836,88,80,80,77,76,66,65,62,61,61,59,58,58,39,39,39,37,37,36,36


In [26]:
test_df[['user_id','0','1','2','3','4','5','6','7','8','9','10','11','12','13','14',\
        '15','16','17','18','19']].to_csv('submission01.csv',index=False,header=True)

In [27]:
!ls


catboost_info  model_recommend_cb01.pickle  submission01.csv
gdrive	       sample_data


In [28]:
from google.colab import files

files.download('submission01.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>